In [190]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import rand_score
from sklearn.cluster import KMeans

In [191]:
test_df = pd.read_csv('Test.csv')
train_df = pd.read_csv('Train.csv')

In [192]:
train_df.head(5)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


## Описание набора данных

Согласно описанию, представленны обезличенные данные пользователей:
- user_id - индитификатор пользователя
- REGION - местоположение клиента
- TENURE - времени с момента заключения контракта
- MONTANT - сумма пополнения
- FREQUENCE_RECH - количество пополнений счета
- REVENUE - ежемесячный доход с клиента
- ARPU_SEGMENT - приведенный ежемесячные поступления за последние 90 дней (сумма поступлений за 90 дней / 3)
- FREQUENCE - количество пополнений счета
- DATA_VOLUME - количество подключений
- ON_NET - звонков внутри сети
- ORANGE - звонков абонентам ORANGE
- TIGO - звонков абонентам  Tigo
- ZONE1 - звонков zones1
- ZONE2 - звонков  zones2
- MRG - клиент ушел
- REGULARITY - время активности за последний 90 дней
- TOP_PACK - наиболее часто используемый пакет услуг
- FREQ_TOP_PACK - количество активаций часто используемого пакета услуг
- CHURN - пользователь ушел(целевая переменная)

## Анализ данных на предмет необходимости

In [193]:
train_df.nunique()

user_id           2154048
REGION                 14
TENURE                  8
MONTANT              6540
FREQUENCE_RECH        123
REVENUE             38114
ARPU_SEGMENT        16535
FREQUENCE              91
DATA_VOLUME         41550
ON_NET               9884
ORANGE               3167
TIGO                 1315
ZONE1                 612
ZONE2                 486
MRG                     1
REGULARITY             62
TOP_PACK              140
FREQ_TOP_PACK         245
CHURN                   2
dtype: int64

Данные в колонке *MRG* на всем объеме обучающих данных имею единственное значение - исключаем.

In [194]:
train_df = train_df.drop(columns=['MRG'])
train_df

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,11,Mixt 250F=Unlimited_call24H,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,NaN,NaN,55,"Data: 200 F=100MB,24H",9.0,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,THIES,K > 24 month,10000.0,11.0,7120.0,2373.0,13.0,NaN,0.0,140.0,13.0,NaN,NaN,28,All-net 500F=2000F;5d,12.0,0


In [195]:
# % пропущенных данных
train_df.isna().sum() / train_df.shape[0] * 100

user_id            0.000000
REGION            39.428044
TENURE             0.000000
MONTANT           35.131018
FREQUENCE_RECH    35.131018
REVENUE           33.706213
ARPU_SEGMENT      33.706213
FREQUENCE         33.706213
DATA_VOLUME       49.229776
ON_NET            36.520774
ORANGE            41.561191
TIGO              59.887988
ZONE1             92.120835
ZONE2             93.648052
REGULARITY         0.000000
TOP_PACK          41.902223
FREQ_TOP_PACK     41.902223
CHURN              0.000000
dtype: float64

Проанализировав процент пропущенных значений для каждого из параметров принимается следующая стратегия:
- REGION - заполнение признаком "other", возможно что заполнение каким-либо средним приведет к искажению результатов;
- MONTANT - наличие пропущенных данных в этом поле говорит, что пополнений счета не было - заполняем значением 0;
- FREQUENCE_RECH - аналогично MONTANT - заполняем значением 0;
- REVENUE - аналогично MONTANT - заполняем значением 0;
- ARPU_SEGMENT - аналогично MONTANT - заполняем значением 0;
- FREQUENCE - аналогично MONTANT - заполняем значением 0;

Следующие параметры отражают активность пользователя в специфической области:
- DATA_VOLUME - принимаем nan как отсутствие активности и присваиваем 0;
- ON_NET - принимаем nan как отсутствие активности и присваиваем 0;
- ORANGE - принимаем nan как отсутствие активности и присваиваем 0;
- TIGO - принимаем nan как отсутствие активности и присваиваем 0;
- ZONE1 - принимаем nan как отсутствие активности и присваиваем 0;
- ZONE2 - принимаем nan как отсутствие активности и присваиваем 0;

Для уменьшения вероятности потери данных, для восполнения отсутствующих значений используем метод кластеризации:
- TOP_PACK - с помощью кластеризации определим возможное значение;
- FREQ_TOP_PACK - после заполнение TOP_PACK с помощью кластеризации определим возможное значение.

In [196]:
# заполняем и переводим в int для оптимизации памяти
train_df["REGION"] = train_df["REGION"].fillna("OTHER")
train_df["MONTANT"] = train_df["MONTANT"].fillna(0)
train_df["FREQUENCE_RECH"] = train_df["FREQUENCE_RECH"].fillna(0)
train_df["REVENUE"] = train_df["REVENUE"].fillna(0)
train_df["ARPU_SEGMENT"] = train_df["ARPU_SEGMENT"].fillna(0)
train_df["FREQUENCE"] = train_df["FREQUENCE"].fillna(0)
train_df["DATA_VOLUME"] = train_df["DATA_VOLUME"].fillna(0)
train_df["ON_NET"] = train_df["ON_NET"].fillna(0)
train_df["ORANGE"] = train_df["ORANGE"].fillna(0)
train_df["TIGO"] = train_df["TIGO"].fillna(0)
train_df["ZONE1"] = train_df["ZONE1"].fillna(0)
train_df["ZONE2"] = train_df["ZONE2"].fillna(0)
train_df

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,OTHER,I 18-21 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,OTHER,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,0.0,90.0,46.0,7.0,0.0,0.0,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,0.0,0.0,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,0.0,39.0,24.0,0.0,0.0,0.0,11,Mixt 250F=Unlimited_call24H,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,OTHER,K > 24 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,NaN,NaN,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,0.0,0.0,55,"Data: 200 F=100MB,24H",9.0,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,OTHER,K > 24 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN,NaN,1
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,THIES,K > 24 month,10000.0,11.0,7120.0,2373.0,13.0,0.0,0.0,140.0,13.0,0.0,0.0,28,All-net 500F=2000F;5d,12.0,0


In [197]:
# оптимизируем типы
train_df['CHURN'] = train_df['CHURN'].astype(bool)
train_df['MONTANT'] = train_df['MONTANT'].astype(np.int32)
train_df['FREQUENCE_RECH'] = train_df['FREQUENCE_RECH'].astype(np.int16)
train_df['REVENUE'] = train_df['REVENUE'].astype(np.int32)
train_df['ARPU_SEGMENT'] = train_df['ARPU_SEGMENT'].astype(np.int32)
train_df['FREQUENCE'] = train_df['FREQUENCE'].astype(np.int16)
train_df['DATA_VOLUME'] = train_df['DATA_VOLUME'].astype(np.int32)
train_df['ON_NET'] = train_df['ON_NET'].astype(np.int32)
train_df['ORANGE'] = train_df['ORANGE'].astype(np.int16)
train_df['TIGO'] = train_df['TIGO'].astype(np.int16)
train_df['ZONE1'] = train_df['ZONE1'].astype(np.int16)
train_df['ZONE2'] = train_df['ZONE2'].astype(np.int16)
train_df['REGULARITY'] = train_df['REGULARITY'].astype(np.int16)

In [198]:
# проверяем что типы данных не "узкие", т.е. после преобразования не появилось отрицательных значений
train_df.describe()

,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
count,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,1.251454e+06
mean,3.588628e+03,7.478823e+00,3.653325e+03,1.217779e+03,9.266639e+00,1.709154e+03,1.762749e+02,5.576156e+01,9.269586e+00,6.437382e-01,4.797823e-01,2.804251e+01,9.272461e+00
std,6.307095e+03,1.202461e+01,6.405449e+03,2.135148e+03,1.366740e+01,9.628098e+03,7.080434e+02,1.636068e+02,4.182922e+01,1.176395e+01,8.638500e+00,2.228686e+01,1.228044e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,2.000000e+00
50%,1.000000e+03,2.000000e+00,1.000000e+03,3.330000e+02,3.000000e+00,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,5.000000e+00
75%,4.600000e+03,1.000000e+01,4.700000e+03,1.567000e+03,1.300000e+01,2.890000e+02,5.500000e+01,4.100000e+01,3.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01,1.200000e+01
max,4.700000e+05,1.330000e+02,5.321770e+05,1.773920e+05,9.100000e+01,1.823866e+06,5.080900e+04,2.132300e+04,4.174000e+03,4.792000e+03,3.697000e+03,6.200000e+01,7.130000e+02


In [199]:
# произведем One Hot кодирование категориальных переменных в столбцах 'TENURE' и 'REGION'
train_df = pd.get_dummies(train_df, columns=['TENURE', 'REGION'])
train_df

,user_id,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,REGION_KEDOUGOU,REGION_KOLDA,REGION_LOUGA,REGION_MATAM,REGION_OTHER,REGION_SAINT-LOUIS,REGION_SEDHIOU,REGION_TAMBACOUNDA,REGION_THIES,REGION_ZIGUINCHOR
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,4250,15,4251,1417,17,4,388,46,1,...,0,0,0,0,0,0,0,0,0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,00001654a9d9f96303d9969d0a4a851714a4bb57,3600,2,1020,340,2,0,90,46,7,...,0,0,0,0,1,0,0,0,0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,13500,15,13502,4501,18,43804,41,102,2,...,0,0,0,0,0,0,0,0,0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,1000,1,985,328,1,0,39,24,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,6100,15,5800,1933,15,621,26,40,40,...,0,0,0,0,0,0,0,0,1,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,10000,11,7120,2373,13,0,0,140,13,...,0,0,0,0,0,0,0,0,1,0


In [200]:
user_ids = train_df['user_id']
train_df = train_df.drop(columns='user_id')
user_ids

0          00000bfd7d50f01092811bc0c8d7b0d6fe7c3596
1          00000cb4a5d760de88fecb38e2f71b7bec52e834
2          00001654a9d9f96303d9969d0a4a851714a4bb57
3          00001dd6fa45f7ba044bd5d84937be464ce78ac2
4          000028d9e13a595abe061f9b58f3d76ab907850f
                             ...                   
2154043    ffffe85215ddc71a84f95af0afb0deeea90e6967
2154044    ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15
2154045    fffff172fda1b4bb38a95385951908bb92379809
2154046    fffff5911296937a37f09a37a549da2e0dad6dbb
2154047    fffff6dbff1508ea2bfe814e5ab2729ce6b788c2
Name: user_id, Length: 2154048, dtype: object

In [201]:
train_df.shape

(2154048, 38)

In [202]:
# выделяем набор данных для предсказания TOP_PACK
top_pack_predict_df = train_df[train_df["TOP_PACK"].isna()].drop(columns=['TOP_PACK', 'FREQ_TOP_PACK'])

In [203]:
# набор данных для обучения и теста метода KNN для дальнейшего предсказания TOP_PACK
# разделение на параметры и целевую переменную
top_pack_traintest_df = train_df[~train_df["TOP_PACK"].isna()]
top_pack_X_df = top_pack_traintest_df.drop(columns=['TOP_PACK', 'FREQ_TOP_PACK'])
top_pack_y_df = top_pack_traintest_df['TOP_PACK']
top_pack_X_df.dtypes

MONTANT                 int32
FREQUENCE_RECH          int16
REVENUE                 int32
ARPU_SEGMENT            int32
FREQUENCE               int16
DATA_VOLUME             int32
ON_NET                  int32
ORANGE                  int16
TIGO                    int16
ZONE1                   int16
ZONE2                   int16
REGULARITY              int16
CHURN                    bool
TENURE_D 3-6 month      uint8
TENURE_E 6-9 month      uint8
TENURE_F 9-12 month     uint8
TENURE_G 12-15 month    uint8
TENURE_H 15-18 month    uint8
TENURE_I 18-21 month    uint8
TENURE_J 21-24 month    uint8
TENURE_K > 24 month     uint8
REGION_DAKAR            uint8
REGION_DIOURBEL         uint8
REGION_FATICK           uint8
REGION_KAFFRINE         uint8
REGION_KAOLACK          uint8
REGION_KEDOUGOU         uint8
REGION_KOLDA            uint8
REGION_LOUGA            uint8
REGION_MATAM            uint8
REGION_OTHER            uint8
REGION_SAINT-LOUIS      uint8
REGION_SEDHIOU          uint8
REGION_TAM

In [204]:
top_pack_y_df

0                     On net 200F=Unlimited _call24H
2                            On-net 1000F=10MilF;10d
3                                  Data:1000F=5GB,7d
4                        Mixt 250F=Unlimited_call24H
5          MIXT:500F= 2500F on net _2500F off net;2d
                             ...                    
2154038                        All-net 500F=2000F;5d
2154040                        All-net 500F=2000F;5d
2154042                           Data:1000F=2GB,30d
2154044                        Data: 200 F=100MB,24H
2154046                        All-net 500F=2000F;5d
Name: TOP_PACK, Length: 1251454, dtype: object

In [205]:
# создаем модель для предсказания TOP_PACK методом кластеризации,
# количество кластеров == кол-ву возможных значений TOP_PACK

cluster_model1 = KMeans(n_clusters=len(train_df['TOP_PACK'].unique()))

top_pack_X_df_train, top_pack_X_df_test, top_pack_y_df_train, top_pack_y_df_test = train_test_split(top_pack_X_df,
                                                                                                    top_pack_y_df,
                                                                                                    train_size=0.7,
                                                                                                    random_state=777)

In [206]:
# проверяем если модельку уже делали, то просто загружаем её, если нет - обучаем
try:
    cluster_model1 = pickle.load(open('TOP_PACK_predict_model.pkl', 'rb'))
except FileNotFoundError:
    cluster_model1.fit(top_pack_X_df_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.1.0 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [207]:
predicted_y_train = cluster_model1.predict(top_pack_X_df_train)
predicted_y_test = cluster_model1.predict(top_pack_X_df_test)

print(
    f"Результаты кластеризации:\nТренировочная выборка: {rand_score(top_pack_y_df_train, predicted_y_train)}\nТестовая выборка: {rand_score(top_pack_y_df_test, predicted_y_test)}")

Результаты кластеризации:
Тренировочная выборка: 0.8637495953662315
Тестовая выборка: 0.8640303389465517


In [208]:
# точность явно лучше случайного распределения, либо назначения среднего значения либо же моды, сохраняем модель в файл для следующего использования:
while True:
    answer = input('Пезезаписать файл модели? y/n')
    if str.lower(answer) == 'y':
        pickle.dump(cluster_model1, open('TOP_PACK_predict_model.pkl', 'wb'))
    else:
        break

In [209]:
# заполняем отсутствующие значения "TOP_PACK"
top_pack_predict_df_values = cluster_model1.predict(top_pack_predict_df)
top_pack_labels = train_df['TOP_PACK'].unique()
top_pack_predict_df['TOP_PACK'] = top_pack_labels[top_pack_predict_df_values]
top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()
train_df['TOP_PACK'] = top_pack_y_df
train_df

/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_49793/1459915616.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()


,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,...,REGION_KEDOUGOU,REGION_KOLDA,REGION_LOUGA,REGION_MATAM,REGION_OTHER,REGION_SAINT-LOUIS,REGION_SEDHIOU,REGION_TAMBACOUNDA,REGION_THIES,REGION_ZIGUINCHOR
0,4250,15,4251,1417,17,4,388,46,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3600,2,1020,340,2,0,90,46,7,0,...,0,0,0,0,1,0,0,0,0,0
3,13500,15,13502,4501,18,43804,41,102,2,0,...,0,0,0,0,0,0,0,0,0,0
4,1000,1,985,328,1,0,39,24,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2154044,6100,15,5800,1933,15,621,26,40,40,0,...,0,0,0,0,0,0,0,0,1,0
2154045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2154046,10000,11,7120,2373,13,0,0,140,13,0,...,0,0,0,0,0,0,0,0,1,0


In [210]:
# проверяем на наличие каких-либо "не определенных" значений
train_df['TOP_PACK'].isna().sum()

1

In [213]:
# удаляем неопределенности
train_df = train_df.dropna(subset=['TOP_PACK'])
train_df['TOP_PACK'].isna().sum()

0

In [233]:
# необходимо осуществить кодирование TOP_PACK, однако делать это наивно не имеет смысл,
# т.к.  при One Hot кодировании получится более 140 дополнительных признаков, что не имеет смысла
# поэтому произведем поиск "уникальных" сущностей в названиях пакетов
packs = list(train_df['TOP_PACK'].unique())
packs_entity = set()
for pack in packs:
    packs_entity.add(*pack.lower().split())
packs_entity

TypeError: set.add() takes exactly one argument (4 given)

In [216]:
# TODO: реализовать аналогично подбор данных для второго проблемного столбца "FREQ_TOP_PACK"
# разделяем набор данных на известные и неизвестные
freq_top_pack_to_train = train_df[~train_df['FREQ_TOP_PACK'].isna()]
freq_top_pack_to_prediction = train_df[train_df['FREQ_TOP_PACK'].isna()]
freq_top_pack_to_train.shape, freq_top_pack_to_prediction.shape

((1251454, 38), (902593, 38))

In [217]:
# подготовка тестовой и тренировочной выборок
freq_top_pack_X_df = freq_top_pack_to_train.drop(columns=['FREQ_TOP_PACK'])
freq_top_pack_y_df = freq_top_pack_to_train['FREQ_TOP_PACK']

freq_top_pack_X_df_train, freq_top_pack_X_df_test, freq_top_pack_y_df_train, freq_top_pack_y_df_test = train_test_split(
    freq_top_pack_X_df,
    freq_top_pack_y_df,
    train_size=0.7,
    random_state=777)

In [218]:
cluster_model2 = KMeans(n_clusters=len(train_df['FREQ_TOP_PACK'].unique()))

In [219]:
#cluster_model2.fit(freq_top_pack_X_df_train, freq_top_pack_y_df_train)

ValueError: could not convert string to float: 'On net 200F=Unlimited _call24H'